# Getting Familiarised with the TLE object: Analysis of a Repeating Sun-Synchronous Orbit

Many Earth observation satellites at Low Earth Orbit (LEO) are in
[Sun-Synchronous Orbit](https://en.wikipedia.org/wiki/Sun-synchronous_orbit), where the orbital
plane of the satellite keeps a fixed (or changing around a mean) orientation with respect to the sun.
This enables the satellite to keep a predictable sun-Earth-satellite orientation, increasing the
data production quality. Some of these satellites also keep the orbit in such a way that the satellite
[ground-track](https://en.wikipedia.org/wiki/Ground_track) repeats a certain pattern. Some are even in
[Frozen Orbits](https://en.wikipedia.org/wiki/Frozen_orbit), where the eccentricity vector (or
argument of perigee value is fixed). This enables the satellite to have a fixed (and known) altitude over a
given latitude. All of these properties enable the satellite orbit (hence observation conditions as well as
mission planning) to be predictable over the course of the mission lifetime.

[European Space Agency's Sentinel-2A](https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-2)
optical earth observation satellite is an example of missions utilising such orbits. It is possible to
observe these orbital characteristics from their orbital elements.

The up-to-date orbital elements of Sentinel-2A can be retrieved from
[Celestrak website](https://celestrak.com/satcat/tle.php?CATNR=40697). The orbital elements are in
[Two-Line-Element format](https://celestrak.com/NORAD/documentation/tle-fmt.php).

The first step of the analysis is to read the orbital elements to a TLE object.

In [23]:
from satmad.propagation.tle import TLE

line1 = "1 40697U 15028A   20164.50828565  .00000010  00000-0  20594-4 0  9999"
line2 = "2 40697  98.5692 238.8182 0001206  86.9662 273.1664 14.30818200259759"
tle = TLE.from_tle(line1, line2, "Sentinel 2A")

print(f"Sentinel-2A true semimajor axis: {tle.sm_axis}")

Sentinel-2A true semimajor axis: 7167.136364515168 km


Unlike ideal (and unrealistic) two-body dynamics, orbital planes of the satellites rotate due to the
primary term of the oblateness of the Earth, or $J_2$ in orbital mechanics parlance. Sun-synchronous orbits
are designed to fix that rotation rate to one full rotation per year i.e, this rotation has a period of
exactly one year. This keeps the orientation of the orbital plane (ideally) at a fixed angle to the Sun.
Therefore, the rate of rotation of the orbital plane (or the node) should be equal to:

$$ \frac{360 \deg}{365.25 \text{ days}} = 0.9856262833675564 \deg / \text{day}$$

Coming back to the orbit of Sentinel-2A, once the orbital elements are read, it is possible to query the
node rotation rate using the command `node_rotation_rate()`.

In [24]:
print(f"Sentinel-2A true node rotation rate: {tle.node_rotation_rate}")

Sentinel-2A true node rotation rate: 0.9870658041317965 deg / d


As expected, the node rotation rate of Sentinel-2A is very close to (but not exactly equal to) the ideal
node rotation rate. In reality, the node rotation rate depends on other forces (like drag and other
geopotential terms) and Sentinel-2 orbit is maintained with high accuracy, executing orbit maintenance
manoeuvres periodically.

It should also be emphasised the TLE orbit parameters are not the high precision values supplied by ESA, but
are the measurements from the US Space Surveillance Network and distributed by the
[18 SPCS](https://www.space-track.org). Therefore, the mathematical models and the orbit determination precision
all play a role in matching the values derived from the simplified analytical models presented here (and the
orbital mechanics textbooks).


The next step is to check the frozen orbit conditions. Orbit theory suggests that, for frozen orbit, the
following conditions should apply:

$$ \text{eccentricity} = -\frac{J_3 R_E \sin{i} }{2 J_2 a} $$
$$ \text{argument of perigee} = \omega =  90 \deg $$

We can easily check this with Sentinel-2A:

In [25]:
import numpy as np
from astropy import units as u

a = tle.sm_axis
e = tle.eccentricity
i = tle.inclination
j2 = 0.001082616
j3 = -0.00000253881
r_e = 6378.135 * u.km

print(f"ideal eccentricity : {-(j3 * r_e * np.sin(i)) / (2 * j2 * a)}" )
print(f"true eccentricity  : {tle.eccentricity}")
print(f"true arg of perigee : {tle.arg_perigee.to(u.deg)}")

ideal eccentricity : 0.0010318067048681323
true eccentricity  : 0.0001206
true arg of perigee : 86.9662 deg


Once again, the results are *close enough*, but not exactly the same. The same caveats apply as before.

This example is limited to the basic uses of the `TLE` object, therefore more advanced analyses will not be
shown.